# Bornholdt Ising Model Simulation Overview

This notebook runs and manages a **Bornholdt-style Ising model** for financial markets, simulating collective market behavior through interacting spins. It uses **Numba-accelerated Monte Carlo sweeps** for speed and saves all results neatly into a `Sim_Datasets/` folder beside the script.

The setup begins by checking for existing simulation outputs, reporting which parameter combinations already exist and which are missing. Each simulation evolves a 2D spin grid with periodic boundaries, where the local field depends on neighboring spins and a global “frozen” magnetization term. After a burn-in phase, magnetization values are recorded, their differences computed as returns, and results saved to compressed `.npz` files with full metadata.

The code sweeps through parameter grids of `J`, `α`, and `β`, skipping any completed runs, ensuring efficiency and reproducibility. A helper function, `load_sim()`, easily reloads data for analysis. Overall, the notebook automates simulation management, data storage, and efficient computation in a clean, reproducible workflow.


In [2]:
# =====================================================
# === Import libraries
# =====================================================

import numpy as np
from pathlib import Path
import os, sys, time
from pathlib import Path
from numba import njit

In [3]:
# =====================================================
# === Dry run: check existing simulation files
# =====================================================

# =====================================================
# === Simulation data directory
# =====================================================
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()

SIM_DIR = BASE_DIR / "Sim_Datasets"
SIM_DIR.mkdir(parents=True, exist_ok=True)

# =====================================================
# === Filename utilities (exact 1-decimal formatting)
# =====================================================
def fmt1(x):
    return f"{float(x):.1f}"   # e.g., 0.5 -> "0.5", 6.0 -> "6.0"

def base_name(J, ALPHA, BETA, seed):
    return f"bornholdt_simulation_J-{fmt1(J)}_Alpha-{fmt1(ALPHA)}_Beta-{fmt1(BETA)}_Seed-{seed}"

def exists_any(J, ALPHA, BETA, seed):
    stem = base_name(J, ALPHA, BETA, seed)
    # Look for .npz and bare file inside Sim_Datasets/
    p_npz = SIM_DIR / f"{stem}.npz"
    p_stem = SIM_DIR / stem
    return p_npz.exists() or p_stem.exists()

# =====================================================
# === Parameter grid (match main script)
# =====================================================
SEED = 42
J_values     = np.arange(0.0, 2.5 + 1e-9, 0.5)
ALPHA_values = np.arange(4.0, 8.0 + 1e-9, 1.0)
BETA_values  = np.arange(0.4, 1.2 + 1e-9, 0.1)
param_list = [(J, A, B) for J in J_values for A in ALPHA_values for B in BETA_values]

# =====================================================
# === Dry run report
# =====================================================
to_run, to_skip = [], []
for (J, A, B) in param_list:
    (to_skip if exists_any(J, A, B, SEED) else to_run).append((J, A, B))

print(f"\n=== Dry Run Check ===")
print(f"Simulation directory: {SIM_DIR}")
print(f"Total parameter combinations: {len(param_list)}")
print(f"Existing files (skip): {len(to_skip)}")
print(f"Missing files (run):  {len(to_run)}\n")

if to_skip:
    print("Already saved:")
    for (J, A, B) in to_skip:
        print(f"  - J={fmt1(J)}, α={fmt1(A)}, β={fmt1(B)}")

if to_run:
    print("\nStill missing:")
    for (J, A, B) in to_run:
        print(f"  - J={fmt1(J)}, α={fmt1(A)}, β={fmt1(B)}")



=== Dry Run Check ===
Simulation directory: c:\Users\remim\OneDrive\Documents\Superieur\Bocconi\Courses\30561 Stochastic Processes\Project\Expectation Bubbles in Spin Model of Markets\Code1\Sim_Datasets
Total parameter combinations: 270
Existing files (skip): 225
Missing files (run):  45

Already saved:
  - J=0.0, α=4.0, β=0.4
  - J=0.0, α=4.0, β=0.5
  - J=0.0, α=4.0, β=0.6
  - J=0.0, α=4.0, β=0.7
  - J=0.0, α=4.0, β=0.8
  - J=0.0, α=4.0, β=0.9
  - J=0.0, α=4.0, β=1.0
  - J=0.0, α=4.0, β=1.1
  - J=0.0, α=4.0, β=1.2
  - J=0.0, α=5.0, β=0.4
  - J=0.0, α=5.0, β=0.5
  - J=0.0, α=5.0, β=0.6
  - J=0.0, α=5.0, β=0.7
  - J=0.0, α=5.0, β=0.8
  - J=0.0, α=5.0, β=0.9
  - J=0.0, α=5.0, β=1.0
  - J=0.0, α=5.0, β=1.1
  - J=0.0, α=5.0, β=1.2
  - J=0.0, α=6.0, β=0.4
  - J=0.0, α=6.0, β=0.5
  - J=0.0, α=6.0, β=0.6
  - J=0.0, α=6.0, β=0.7
  - J=0.0, α=6.0, β=0.8
  - J=0.0, α=6.0, β=0.9
  - J=0.0, α=6.0, β=1.0
  - J=0.0, α=6.0, β=1.1
  - J=0.0, α=6.0, β=1.2
  - J=0.0, α=7.0, β=0.4
  - J=0.0, α=7.0, β=0.

In [4]:
# NUMBA ADJUSTED VERSION (with robust filename handling + Sim_Datasets dir)
# ------------------------------------------------------------------------

# =====================================================
# === Simulation configuration
# =====================================================
N = 32
STEPS = 100_000       # total sweeps per simulation
BURN_IN = 10_000      # burn-in sweeps (not recorded)
SEED = 42             # fixed seed for reproducibility

# =====================================================
# === Simulation data folder (Sim_Datasets next to this file)
# =====================================================
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    # Fallback for interactive runs
    BASE_DIR = Path.cwd()

SIM_DIR = BASE_DIR / "Sim_Datasets"
SIM_DIR.mkdir(parents=True, exist_ok=True)

# =====================================================
# === Numba-compiled kernels
# =====================================================
@njit
def neighbors_sum_numba(spins, i, j):
    n = spins.shape[0]
    up    = spins[(i-1) % n, j]
    down  = spins[(i+1) % n, j]
    left  = spins[i, (j-1) % n]
    right = spins[i, (j+1) % n]
    return up + down + left + right

@njit
def sweep_frozenM_kernel(spins, J, alpha, beta, M_frozen, order_flat, uvals):
    """
    One asynchronous sweep with magnetization frozen at M_frozen.
    order_flat : 1D array of flattened lattice indices.
    uvals      : pre-generated uniform randoms (0–1).
    """
    n = spins.shape[0]
    for k in range(order_flat.size):
        idx = order_flat[k]
        i = idx // n
        j = idx % n
        s = spins[i, j]
        h_local = J * neighbors_sum_numba(spins, i, j)
        h = h_local - alpha * s * abs(M_frozen)
        p = 1.0 / (1.0 + np.exp(-2.0 * beta * h))
        spins[i, j] = 1 if uvals[k] < p else -1

# =====================================================
# === Filename utilities (exact 1-decimal formatting) in SIM_DIR
# =====================================================
def fmt1(x):
    return f"{float(x):.1f}"   # e.g., 0.5 -> "0.5", 6.0 -> "6.0"

def make_basename(J, ALPHA, BETA, seed):
    return f"bornholdt_simulation_J-{fmt1(J)}_Alpha-{fmt1(ALPHA)}_Beta-{fmt1(BETA)}_Seed-{seed}"

def existing_file_path(J, ALPHA, BETA, seed):
    """
    Return Path to an existing file if found (prefer .npz), else None.
    Checks both the bare stem (no extension) and '.npz' inside SIM_DIR.
    """
    stem = make_basename(J, ALPHA, BETA, seed)
    p_npz = SIM_DIR / (stem + ".npz")
    p_stem = SIM_DIR / stem
    if p_npz.exists():
        return p_npz
    if p_stem.exists():
        return p_stem
    return None

def target_save_path(J, ALPHA, BETA, seed):
    """Always save to the canonical .npz filename inside SIM_DIR."""
    return SIM_DIR / (make_basename(J, ALPHA, BETA, seed) + ".npz")

# =====================================================
# === Single simulation routine
# =====================================================
def run_and_save_sim(J, ALPHA, BETA, seed=SEED):
    # Skip if either stem or '.npz' already exists
    already = existing_file_path(J, ALPHA, BETA, seed)
    if already is not None:
        print(f"[SKIP] {already.name} already exists in {SIM_DIR.name}/")
        return str(already)

    filename = target_save_path(J, ALPHA, BETA, seed)
    print(f"\n[RUN ] Grid={N}x{N}, Steps={STEPS}, Burn-in={BURN_IN}, "
          f"J={fmt1(J)}, α={fmt1(ALPHA)}, β={fmt1(BETA)}, Seed={seed}")
    print(f"[DIR ] Saving to: {SIM_DIR}")

    rng = np.random.default_rng(seed)
    spins = rng.choice(np.array([-1, 1], dtype=np.int8), size=(N, N))
    M_values = []
    order_flat = np.arange(N * N, dtype=np.int64)
    tick = max(1, int(STEPS * 0.01))  # every 1%

    # Warm up Numba (first compile)
    dummy_u = rng.random(order_flat.size)
    M0 = spins.mean()
    rng.shuffle(order_flat)
    sweep_frozenM_kernel(spins, J, ALPHA, BETA, M0, order_flat, dummy_u)

    t0 = time.time()
    for t in range(STEPS):
        M_frozen = spins.mean()
        rng.shuffle(order_flat)
        uvals = rng.random(order_flat.size)
        sweep_frozenM_kernel(spins, J, ALPHA, BETA, M_frozen, order_flat, uvals)

        if t >= BURN_IN:
            M_values.append(spins.mean())

        if (t + 1) % tick == 0 or (t + 1) == STEPS:
            pct = (t + 1) / STEPS * 100
            sys.stdout.write(f"\rProgress [{filename.name}]: {pct:6.2f}%")
            sys.stdout.flush()

    sys.stdout.write("\n")
    dt = time.time() - t0
    print(f"[DONE] {filename.name} in {dt:.1f}s")

    M_values = np.array(M_values, dtype=float)
    returns = np.diff(M_values)

    np.savez_compressed(
        filename,
        M_values=M_values,
        returns=returns,
        N=N,
        STEPS=STEPS,
        BURN_IN=BURN_IN,
        J=float(J),
        ALPHA=float(ALPHA),
        BETA=float(BETA),
        SEED=int(seed)
    )
    print(f"[SAVE] {filename.name} -> {filename.resolve()}")
    return str(filename)

# Optional helper to load from SIM_DIR by default
def load_sim(J, ALPHA, BETA, seed=SEED):
    path = existing_file_path(J, ALPHA, BETA, seed)
    if path is None:
        raise FileNotFoundError(f"No simulation found for J={J}, α={ALPHA}, β={BETA}, seed={seed} in {SIM_DIR}")
    return np.load(path)

# =====================================================
# === Parameter grid (edit as you like)
# =====================================================
SEED = 42
J_values     = np.arange(0.0, 2.5 + 1e-9, 0.5)
ALPHA_values = np.arange(4.0, 8.0 + 1e-9, 1.0)
BETA_values  = np.arange(0.4, 1.2 + 1e-9, 0.1)
param_list = [(J, a, b) for J in J_values for a in ALPHA_values for b in BETA_values]

# =====================================================
# === Batch run
# =====================================================
if __name__ == "__main__":
    all_files = []
    for (J, ALPHA, BETA) in param_list:
        try:
            fname = run_and_save_sim(J, ALPHA, BETA, seed=SEED)
            all_files.append(fname)
        except Exception as e:
            print(f"[ERR ] J={fmt1(J)}, α={fmt1(ALPHA)}, β={fmt1(BETA)} -> {e}")

    print("\nBatch complete. Files:")
    for f in all_files:
        print(" -", f)


[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-0.4_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-0.5_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-0.6_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-0.7_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-0.8_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-0.9_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-1.0_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-1.1_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-4.0_Beta-1.2_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-0.0_Alpha-5.0_Beta-0.4_Seed-42.npz already exists in Sim_Datasets/
[SKIP] bornholdt_simulation_J-

KeyboardInterrupt: 